<a href="https://colab.research.google.com/github/MrKelv/full-tensorflow-rock-paper-scissors/blob/main/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
data, info = tfds.load(name='rock_paper_scissors',
                       split=['train','test'],
                       as_supervised=True,
                       with_info=True)

In [3]:
train, test = data

In [4]:
print(f"Data Shape : {info.features['image'].shape}")
print(f"Classifier : {info.features['label'].num_classes}")

Data Shape : (300, 300, 3)
Classifier : 3


In [5]:
print(f"Banyak Data Train : {info.splits['train'].num_examples}")
print(f"Banyak Data Test : {info.splits['test'].num_examples}")

Banyak Data Train : 2520
Banyak Data Test : 372


In [6]:
print(type(train))
print(type(test))

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


In [7]:
training_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2
)